In [49]:
import numpy as np
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
# import xgboost as xgb
# import lightgbm as lgb
# from catboost import CatBoostClassifier
# from tpot import TPOTClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import Lasso
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import BayesianRidge

In [2]:
# Connect to SQLite database and retrieve data
conn = sqlite3.connect('wine_quality.db')
query = "SELECT * FROM wine_quality"
wine_df = pd.read_sql_query(query, conn)
conn.close()

In [3]:
print("Data has been successfully retrieved from wine_quality.db")

Data has been successfully retrieved from wine_quality.db


In [4]:
# Set up a binary identifier for quality
wine_df['quality'] = wine_df['quality'].apply(lambda x: 1 if x >= 7 else 0)
wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0,Red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0,Red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0,Red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,0,Red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0,Red


In [5]:
# Count how many "good" wines
good_wines = wine_df[wine_df['quality'] == 1]
quality_counts = good_wines['quality'].count()
print(quality_counts)

1277


In [6]:
# Separate features and target variable
X = wine_df.drop(['quality', 'type'], axis=1)
y = wine_df['quality']

In [7]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [50]:
# Function to evaluate and print model performance
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return classification_report(y_test, y_pred, output_dict=True), confusion_matrix(y_test, y_pred), accuracy

In [51]:
models = [
    ('Logistic Regression', LogisticRegression(random_state=42)),
    ('Linear Discriminant Analysis', LinearDiscriminantAnalysis()),
    ('Support Vector Machine', SVC(random_state=42)),
    ('Decision Tree Classifier', DecisionTreeClassifier(random_state=42)),
    ('Random Forest Classifier', RandomForestClassifier(random_state=42)),
    ('Gradient Boosting Classifier', GradientBoostingClassifier(random_state=42)),
    # ('XGBoost', xgb.XGBClassifier(random_state=42)),
    # ('LightGBM', lgb.LGBMClassifier(random_state=42)),
    # ('CatBoost', CatBoostClassifier(verbose=0, random_state=42)),
    ('AdaBoost', AdaBoostClassifier(random_state=42)),
    ('Bagging Classifier', BaggingClassifier(random_state=42)),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Gaussian Naive Bayes', GaussianNB()),
    ('Quadratic Discriminant Analysis', QuadraticDiscriminantAnalysis()),
    ('Multilayer Perceptron', MLPClassifier(random_state=42)),
    ('Ridge Classifier', RidgeClassifier(random_state=42)),
    ('ExtraTrees Classifier', ExtraTreesClassifier(random_state=42)),
    ('Isolation Forest', IsolationForest(random_state=42))
    # ('PCA', make_pipeline(StandardScaler(), PCA(n_components=2))),
    # ('K-Means Clustering', make_pipeline(StandardScaler(), KMeans(n_clusters=2))),
    # ('Bayesian Logistic Regression', BayesianRidge()),
    # ('Lasso', Lasso(random_state=42))
    # ('TPOT Classifier', TPOTClassifier(generations=5, population_size=50, verbosity=2, random_state=42))
]

In [52]:
# DataFrame to store results
results_df = pd.DataFrame(columns=[
    'Model', 'Precision', 'Recall', 'F1-Score', 'Support', 'Accuracy',
    'Predicted Positive Actual Positive', 'Predicted Positive Actual Negative',
    'Predicted Negative Actual Positive', 'Predicted Negative Actual Negative'
])

In [53]:
# Train and evaluate each model
for name, model in models:
    print(f"Evaluating {name}...")
    try:
        if 'PCA' in name or 'K-Means' in name:
            model.fit(X_train)
            y_pred = model.predict(X_test)
            y_pred = (y_pred > 0.5).astype(int) if 'PCA' in name else y_pred
            report = classification_report(y_test, y_pred, output_dict=True)
            cm = confusion_matrix(y_test, y_pred)
            accuracy = accuracy_score(y_test, y_pred)
        else:
            model.fit(X_train, y_train)
            report, cm, accuracy = evaluate_model(model, X_test, y_test)

        # Store the results in the DataFrame
        results_df = results_df.append({
            'Model': name,
            'Precision': report['1']['precision'],
            'Recall': report['1']['recall'],
            'F1-Score': report['1']['f1-score'],
            'Support': report['1']['support'],
            'Accuracy': accuracy,
            'Predicted Positive Actual Positive': cm[1, 1],
            'Predicted Positive Actual Negative': cm[0, 1],
            'Predicted Negative Actual Positive': cm[1, 0],
            'Predicted Negative Actual Negative': cm[0, 0]
        }, ignore_index=True)
    except Exception as e:
        print(f"Failed to evaluate {name}: {e}")


Evaluating Logistic Regression...
Evaluating Linear Discriminant Analysis...
Evaluating Support Vector Machine...


C:\Users\morga\AppData\Local\Temp\ipykernel_21476\3830747791.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\morga\AppData\Local\Temp\ipykernel_21476\3830747791.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\morga\AppData\Local\Temp\ipykernel_21476\3830747791.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\morga\AppData\Local\Temp\ipykernel_21476\3830747791.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


Evaluating Decision Tree Classifier...
Evaluating Random Forest Classifier...


C:\Users\morga\AppData\Local\Temp\ipykernel_21476\3830747791.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


Evaluating Gradient Boosting Classifier...


...

C:\Users\morga\anaconda3\envs\dev\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\morga\AppData\Local\Temp\ipykernel_21476\3830747791.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\morga\AppData\Local\Temp\ipykernel_21476\3830747791.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


Evaluating Ridge Classifier...
Evaluating ExtraTrees Classifier...


C:\Users\morga\AppData\Local\Temp\ipykernel_21476\3830747791.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


Evaluating Isolation Forest...


C:\Users\morga\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\morga\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\morga\anaconda3\envs\dev\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\morga\anaconda3\envs\dev\li

In [54]:
results_df

,Model,Precision,Recall,F1-Score,Support,Accuracy,Predicted Positive Actual Positive,Predicted Positive Actual Negative,Predicted Negative Actual Positive,Predicted Negative Actual Negative
0,Logistic Regression,0.607143,0.269841,0.373626,252.0,0.824615,68,44,184,1004
1,Linear Discriminant Analysis,0.603053,0.313492,0.412533,252.0,0.826923,79,52,173,996
2,Support Vector Machine,0.708333,0.269841,0.390805,252.0,0.836923,68,28,184,1020
3,Decision Tree Classifier,0.574468,0.642857,0.606742,252.0,0.838462,162,120,90,928
4,Random Forest Classifier,0.789189,0.579365,0.668192,252.0,0.888462,146,39,106,1009
5,Gradient Boosting Classifier,0.655172,0.376984,0.478589,252.0,0.840769,95,50,157,998
6,AdaBoost,0.669231,0.345238,0.455497,252.0,0.840000,87,43,165,1005
7,Bagging Classifier,0.736842,0.555556,0.633484,252.0,0.875385,140,50,112,998
8,K-Nearest Neighbors,0.587629,0.452381,0.511211,252.0,0.832308,114,80,138,968
9,Gaussian Naive Bayes,0.422977,0.642857,0.510236,252.0,0.760769,162,221,90,827


In [13]:
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
nn_model.fit(X_train, y_train, epochs=50, batch_size=10, validation_data=(X_test, y_test))

Epoch 1/50
520/520 [==============================] - 2s 2ms/step - loss: 0.4053 - accuracy: 0.8126 - val_loss: 0.3667 - val_accuracy: 0.8292
Epoch 2/50
520/520 [==============================] - 1s 2ms/step - loss: 0.3665 - accuracy: 0.8309 - val_loss: 0.3610 - val_accuracy: 0.8200
Epoch 3/50
520/520 [==============================] - 1s 2ms/step - loss: 0.3565 - accuracy: 0.8332 - val_loss: 0.3479 - val_accuracy: 0.8400
Epoch 4/50
520/520 [==============================] - 1s 1ms/step - loss: 0.3474 - accuracy: 0.8368 - val_loss: 0.3463 - val_accuracy: 0.8315
Epoch 5/50
520/520 [==============================] - 1s 1ms/step - loss: 0.3417 - accuracy: 0.8422 - val_loss: 0.3453 - val_accuracy: 0.8338
Epoch 6/50
520/520 [==============================] - 1s 1ms/step - loss: 0.3347 - accuracy: 0.8439 - val_loss: 0.3494 - val_accuracy: 0.8300
Epoch 7/50
520/520 [==============================] - 1s 2ms/step - loss: 0.3306 - accuracy: 0.8445 - val_loss: 0.3618 - val_accuracy: 0.8300
Epoch 

In [14]:
# Evaluate Neural Network Model
y_pred_nn = (nn_model.predict(X_test) > 0.5).astype("int32")
print(confusion_matrix(y_test, y_pred_nn))
print(classification_report(y_test, y_pred_nn))# Evaluate Neural Network Model
y_pred_nn = (nn_model.predict(X_test) > 0.5).astype("int32")
print(confusion_matrix(y_test, y_pred_nn))
print(classification_report(y_test, y_pred_nn))

41/41 [==============================] - 0s 1ms/step
[[996  52]
 [123 129]]
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      1048
           1       0.71      0.51      0.60       252

    accuracy                           0.87      1300
   macro avg       0.80      0.73      0.76      1300
weighted avg       0.86      0.87      0.86      1300

41/41 [==============================] - 0s 970us/step
[[996  52]
 [123 129]]
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      1048
           1       0.71      0.51      0.60       252

    accuracy                           0.87      1300
   macro avg       0.80      0.73      0.76      1300
weighted avg       0.86      0.87      0.86      1300

